<div style='background-color: #ffc154;
    border: 0.5em solid black;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Devoir maison</h2>
    <h1>Casse-briques</h1>
</div>

<img src='https://ntoulzac.github.io/Cours-NSI-Terminale/devoirs/images/casse_briques.png' width='50%'>

<div style='background-color: #ffc154;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Ebauche du jeu</h2>
</div>

On donne l'ébauche du code d'un jeu de casse-briques, implémenté via le module `pygame` à l'aide de quatre classes `Balle`, `Raquette`, `Brique` et `Jeu`.

Ecrire les spécifications manquantes puis lister, pour chaque classe, les attributs d'instance en précisant leur type et leur rôle.

In [ ]:
import pygame
import pygame.freetype
from math import cos, sin, radians

In [ ]:
class Balle:
    def vitesse_par_angle(self, angle):
        """
        Détermine les composantes horizontale et verticale du vecteur vitesse de la balle.
        """
        self.vx = self.vitesse * cos(radians(angle))
        self.vy = -self.vitesse * sin(radians(angle))

    def __init__(self):
        self.x, self.y = (0, 0)
        self.vitesse = 8
        self.vitesse_par_angle(60)
        self.sur_raquette = True

    def afficher(self, fenetre):
        """
        Spécification à écrire...
        """
        pygame.draw.rect(fenetre, BLANC,
                         (int(self.x-RAYON_BALLE), int(self.y-RAYON_BALLE), 2*RAYON_BALLE, 2*RAYON_BALLE))

    def rebond_raquette(self, raquette):
        """
        Provoque un rebond de la balle en modifiant son vecteur vitesse, lors d'un contact avec la raquette.
        - Entrée : raquelle (instance de la classe Raquette)
        """
        delta_x = raquette.x - self.x
        longueur_totale = raquette.longueur/2 + RAYON_BALLE
        angle = 90 + 80*delta_x/longueur_totale # L'angle du rebond dépend d'où le contact a lieu
        self.vitesse_par_angle(angle)

    def deplacer(self, raquette):
        """
        Spécification à écrire...
        """
        if self.sur_raquette:
            self.y = raquette.y - 2*RAYON_BALLE
            self.x = raquette.x
        else:
            self.x = self.x + self.vx
            self.y = self.y + self.vy
            if raquette.collision_balle(self) and self.vy > 0:
                self.rebond_raquette(raquette)
            if self.x + RAYON_BALLE > XMAX:
                self.vx = -self.vx
            if self.x - RAYON_BALLE < XMIN:
                self.vx = -self.vx
            if self.y + RAYON_BALLE > YMAX:
                self.sur_raquette = True
            if self.y - RAYON_BALLE < YMIN:
                self.vy = -self.vy

In [ ]:
class Raquette:
    def __init__(self):
        self.x = (XMIN + XMAX) / 2
        self.y = YMAX - RAYON_BALLE
        self.longueur = 10*RAYON_BALLE

    def afficher(self, fenetre):
        """
        Spécification à écrire...
        """
        pygame.draw.rect(fenetre, BLANC,
                         (int(self.x-self.longueur/2), int(self.y-RAYON_BALLE), self.longueur, 2*RAYON_BALLE))

    def deplacer(self, x):
        """
        Spécification à écrire...
        """
        if x - self.longueur/2 < XMIN:
            self.x = XMIN + self.longueur/2
        elif x + self.longueur/2 > XMAX:
            self.x = XMAX - self.longueur/2
        else:
            self.x = x

    def collision_balle(self, balle):
        """
        Détermine si la balle entre en collision avec la raquette.
        - Entrée : balle (instance de la classe Balle)
        - Sortie : (booléen, True si la balle touche la raquette, False sinon)
        """
        vertical = abs(self.y - balle.y) < 2*RAYON_BALLE
        horizontal = abs(self.x - balle.x) < self.longueur/2 + RAYON_BALLE
        return vertical and horizontal

In [ ]:
class Brique:
    def __init__(self, position):
        """
        Crée une instance de la classe Brique.
        - Entrées : position (couple de nombres, coordonnées du centre de la brique)
        """
        self.x, self.y = position
        self.vie = 1
        self.largeur = LARGEUR_BRIQUE
        self.hauteur = HAUTEUR_BRIQUE

    def est_en_vie(self):
        """
        Spécification à écrire...
        """
        return self.vie > 0

    def afficher(self, fenetre):
        """
        Spécification à écrire...
        """
        pygame.draw.rect(fenetre, BLANC,
                         (int(self.x-self.largeur/2), int(self.y-self.hauteur/2), self.largeur, self.hauteur))

    def collision_balle(self, balle):
        """
        Détermine si la balle entre en collision avec la brique.
        - Entrée : balle (instance de la classe Balle)
        - Sortie : (booléen, True si la balle touche la brique, False sinon)
        Attention : en cas de collision, la vitesse de la balle est modifiée et la balle rebondit.
        """
        # On part du principe que la largeur de la brique est supérieure à sa hauteur
        marge = self.hauteur/2 + RAYON_BALLE
        delta_y = balle.y - self.y
        touche = False
        if balle.x >= self.x: # Si la balle est à droite de la brique
            delta_x = balle.x - (self.x + self.largeur/2 - self.hauteur/2)
            if abs(delta_y) <= marge and delta_x <= marge: # La balle touche la brique...
                touche = True
                if delta_x <= abs(delta_y): # ... sur le bord haut ou le bord bas...
                    balle.vy = -balle.vy
                else: # ... ou sur le bord droit
                    balle.vx = -balle.vx
        else: # Sinon la balle est à gauche de la brique
            delta_x = balle.x - (self.x - self.largeur/2 + self.hauteur/2)
            if abs(delta_y) <= marge and -delta_x <= marge: # La balle touche la brique...
                touche = True
                if -delta_x <= abs(delta_y): # ... sur le bord haut ou le bord bas...
                    balle.vy = -balle.vy
                else: # ... ou sur le bord gauche
                    balle.vx = -balle.vx
        if touche:
            self.vie -= 1
        return touche

In [ ]:
class Jeu:
    def __init__(self):
        """
        Crée une instance de la classe Jeu.
        """
        self.balle = Balle()
        self.raquette = Raquette()
        self.brique = Brique((400, 100)) # Une seule brique pour le moment dans le jeu...
        self.continuer = True

    def gerer_evenements(self):
        """
        Permet la gestion des actions du joueur (clics, appuis sur une touche, etc.)
        Le jeu se termine en cas de clic sur la croix de fermeture de la fenêtre.
        La balle se lance en cas de clic sur le bouton gauche de la souris, si elle est sur la raquette.
        """
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.continuer = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1: # Si le bouton gauche de la souris est cliqué
                    if self.balle.sur_raquette:
                        self.balle.sur_raquette = False
                        self.balle.vitesse_par_angle(60)

    def mettre_a_jour(self):
        """
        Spécification à écrire...
        """
        x, y = pygame.mouse.get_pos()
        self.balle.deplacer(self.raquette)
        if self.brique.est_en_vie():
            self.brique.collision_balle(self.balle)
        self.raquette.deplacer(x)

    def ajouter_texte(self, texte, position):
        """
        Spécification à écrire...
        """
        texte, rect = police.render(texte, BLANC, size=16)
        rect.center = position
        fenetre.blit(texte, rect)

    def afficher(self, fenetre):
        """
        Spécification à écrire...
        """
        fenetre.fill(NOIR)
        self.ajouter_texte('Amusez-vous bien !', (largeur/2, hauteur/2))
        self.balle.afficher(fenetre)
        self.raquette.afficher(fenetre)
        if self.brique.est_en_vie():
            self.brique.afficher(fenetre)
        pygame.display.flip()

In [ ]:
# Programme principal

pygame.init()

pygame.freetype.init()
police = pygame.freetype.SysFont('Arial', 20)

largeur, hauteur = 800, 600

fenetre = pygame.display.set_mode((largeur, hauteur))
pygame.display.set_caption('Casse-briques')

BLANC = (255, 255, 255)
NOIR = (0, 0, 0)

XMIN, XMAX = 0, largeur
YMIN, YMAX = 0, hauteur
RAYON_BALLE = 10
LARGEUR_BRIQUE = (XMAX - XMIN) / 10
HAUTEUR_BRIQUE = (YMAX - YMIN) / 20

jeu = Jeu()

horloge = pygame.time.Clock() # permet de limiter le nombre de passages dans la boucle chaque seconde
while jeu.continuer:
    jeu.gerer_evenements()
    jeu.mettre_a_jour()
    jeu.afficher(fenetre)
    horloge.tick(50) # max 50 images par seconde
    
pygame.quit()

<div style='background-color: #ffc154;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Améliorations</h2>
</div>

Recopier et modifier le code ci-dessus dans le but d'obtenir les améliorations suivantes :

***Améliorations obligatoires***

- Faire apparaître plusieurs briques, de différentes couleurs, au début du jeu.
- Faire en sorte que le nombre de balles disponibles soit limité, et que le jeu s'arrête lorsque toutes les balles sont perdues. Afficher le nombre de vies restantes à l'écran.
- Gagner un point à chaque fois qu'une brique est touchée. Afficher le score à l'écran.
- Gagner une vie, c'est-à-dire une balle supplémentaire, lorsqu'un certain nombre de points est atteint.
- Augmenter la résistance des briques, qui pourront avoir besoin d'être frappées plusieurs fois avant d'être détruites. Le nombre de coups nécessaires pour détruire la brique devra apparaître en son centre.
- Augmenter de temps en temps la vitesse de la balle.

***Améliorations optionnelles***

- Afficher un message de *Game over* lorsque les vies sont épuisées.
- Prévoir différents niveaux de jeu. Lorsque toutes les briques sont détruites, on passe au niveau supérieur. Les niveaux successifs peuvent être stockés, par exemple, dans un fichier texte.
- Permettre un contrôle de la raquette au clavier.
- Imaginer que quelques briques puissent être mobiles.
- Imaginer que quelques briques puissent contenir un bonus permettant provisoirement, par exemple, de rallonger la raquette, ou de ralentir la balle, ou de détruire les briques sans rebondir dessus...
- Modifier la gestion des rebonds dans le cas où plusieurs briques sont touchées en même temps.
- Ajouter un timer.
- ...

In [ ]:
import pygame
import pygame.freetype
from math import cos, sin, radians
from random import randint

In [ ]:
class Balle:
    def vitesse_par_angle(self, angle):
        self.vx = self.vitesse * cos(radians(angle))
        self.vy = -self.vitesse * sin(radians(angle))

    def __init__(self):
        self.x, self.y = (0, 0)
        self.vitesse = 8
        self.vitesse_par_angle(60) # couple, composantes horizontale et verticale du vecteur vitesse
        self.sur_raquette = True
        self.vie = 5
        self.score = 0

    def afficher(self, fenetre):
        pygame.draw.rect(fenetre, BLANC, (int(self.x-RAYON_BALLE), int(self.y-RAYON_BALLE), 2*RAYON_BALLE, 2*RAYON_BALLE))

    def rebond_raquette(self, raquette):
        delta_x = raquette.x - self.x
        longueur_totale = raquette.longueur/2 + RAYON_BALLE
        angle = 90 + 80 * delta_x/longueur_totale
        self.vitesse_par_angle(angle)

    def deplacer(self, raquette):
        if self.sur_raquette:
            self.y = raquette.y - 2*RAYON_BALLE
            self.x = raquette.x
        else:
            self.x += self.vx
            self.y += self.vy
            if raquette.collision_balle(self) and self.vy > 0:
                self.rebond_raquette(raquette)
            if self.x + RAYON_BALLE > XMAX:
                self.vx = -self.vx
            if self.x - RAYON_BALLE < XMIN:
                self.vx = -self.vx
            if self.y + RAYON_BALLE > YMAX:
                self.sur_raquette = True
                self.vie -= 1
            if self.y - RAYON_BALLE < YMIN:
                self.vy = -self.vy

class Raquette:
    def __init__(self):
        self.x = (XMIN+XMAX)/2
        self.y = YMAX - RAYON_BALLE
        self.longueur = 10*RAYON_BALLE

    def afficher(self, fenetre):
        pygame.draw.rect(fenetre, BLANC, (int(self.x-self.longueur/2), int(self.y-RAYON_BALLE), self.longueur, 2*RAYON_BALLE))

    def deplacer(self, x):
        if x - self.longueur/2 < XMIN:
            self.x = XMIN + self.longueur/2
        elif x + self.longueur/2 > XMAX:
            self.x = XMAX - self.longueur/2
        else:
            self.x = x

    def collision_balle(self, balle):
        vertical = abs(self.y - balle.y) < 2*RAYON_BALLE
        horizontal = abs(self.x - balle.x) < self.longueur/2 + RAYON_BALLE
        return vertical and horizontal

class Brique:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.vie = randint(1, 6)
        self.largeur = LARGEUR_BRIQUE
        self.hauteur = HAUTEUR_BRIQUE

    def en_vie(self):
        return self.vie > 0

    def ajouter_texte(self, texte, position):
        texte, rect = police.render(texte, NOIR, size=16)
        rect.center = position
        fenetre.blit(texte, rect)

    def afficher(self, fenetre):
        pygame.draw.rect(fenetre, COULEURS[self.vie-1], (int(self.x-self.largeur/2)+1, int(self.y-self.hauteur/2)+1, self.largeur-2, self.hauteur-2))
        texte, rect = police.render(str(self.vie), NOIR, size=16)
        rect.center = (self.x, self.y)
        fenetre.blit(texte, rect)

    def collision_balle(self, balle):
        # on suppose que largeur < longueur
        marge = self.hauteur/2 + RAYON_BALLE
        dy = balle.y - self.y
        touche = False
        if balle.x >= self.x: # on regarde a droite
            dx = balle.x - (self.x + self.largeur/2 - self.hauteur/2)
            if abs(dy) <= marge and dx <= marge: # on touche
                touche = True
                if dx <= abs(dy):
                    balle.vy = -balle.vy
                else: # a droite
                    balle.vx = -balle.vx
        else: # on regarde a gauche
            dx = balle.x - (self.x - self.largeur/2 + self.hauteur/2)
            if abs(dy) <= marge and -dx <= marge: # on touche
                touche = True
                if -dx <= abs(dy):
                    balle.vy = -balle.vy
                else: # a gauche
                    balle.vx = -balle.vx
        if touche:
            self.vie -= 1
            balle.score += 1
        return touche

class Jeu:
    def __init__(self):
        self.balle = Balle()
        self.raquette = Raquette()
        self.briques = [Brique(LARGEUR_BRIQUE*(C+0.5), HAUTEUR_BRIQUE*(L+2.5))
                        for C in range(10) for L in range(5)]
        self.continuer = True

    def gerer_evenements(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.continuer = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1: # Si le bouton gauche de la souris a été cliqué
                    if self.balle.sur_raquette:
                        self.balle.sur_raquette = False
                        self.balle.vitesse_par_angle(60)
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    pygame.image.save(fenetre, 'image.png')
                    
    def mettre_a_jour(self):
        x, y = pygame.mouse.get_pos()
        self.balle.deplacer(self.raquette)
        for brique in self.briques:
            if brique.en_vie():
                brique.collision_balle(self.balle)
        self.raquette.deplacer(x)
        if self.balle.vie <= 0:
            self.continuer = False

    def ajouter_texte(self, texte, position):
        texte, rect = police.render(texte, BLANC, size=16)
        rect.center = position
        fenetre.blit(texte, rect)

    def afficher(self, fenetre):
        fenetre.fill(NOIR)
        if self.balle.vie > 1:
            pluriel_balle = 's'
        else:
            pluriel_balle = ''
        self.ajouter_texte(f'Balle{pluriel_balle} restante{pluriel_balle} : {self.balle.vie}', (largeur/3, HAUTEUR_BRIQUE))
        self.ajouter_texte(f'Score : {self.balle.score}', (2*largeur/3, HAUTEUR_BRIQUE))
        self.balle.afficher(fenetre)
        self.raquette.afficher(fenetre)
        for brique in self.briques:
            if brique.en_vie():
                brique.afficher(fenetre)
        pygame.display.flip()

In [ ]:
pygame.init()

pygame.freetype.init()
police = pygame.freetype.SysFont('Arial', 20) # Texte de taille 20

largeur, hauteur = 800, 600 # Dimensions de la fenetre

fenetre = pygame.display.set_mode((largeur, hauteur))
pygame.display.set_caption('Casse briques')

BLANC = (255, 255, 255)
NOIR = (0, 0, 0)
ROUGE = (228, 3, 3)
ORANGE = (255, 140, 0)
JAUNE = (255, 237, 0)
VERT = (0, 128, 38)
BLEU = (0, 77, 255)
VIOLET = (117, 7, 135)
COULEURS = [ROUGE, ORANGE, JAUNE, VERT, BLEU, VIOLET]

XMIN, XMAX = 0, largeur
YMIN, YMAX = 0, hauteur
RAYON_BALLE = 10
LARGEUR_BRIQUE = (XMAX - XMIN) // 10
HAUTEUR_BRIQUE = (YMAX - YMIN) // 20

jeu = Jeu()

clock = pygame.time.Clock()
while jeu.continuer:
    jeu.gerer_evenements()
    jeu.mettre_a_jour()
    jeu.afficher(fenetre)
    clock.tick(50) # max 50 images par seconde

pygame.quit()